In [0]:
%run /Users/veenabhyrava2000@gmail.com/streams/wordcount/03-invoice

In [0]:
class testInvoice():
    def __init__(self):
        self.base_dir="/FileStore/project1"
    def cleanTests(self):
        print("directory cleaning started.....",end='')
        spark.sql("drop table if exists invoice_lineItem_table")
        dbutils.fs.rm("/user/hive/warehouse/invoice_lineItem_table", True)
        dbutils.fs.rm(f"{self.base_dir}/data/invoices", True)
        dbutils.fs.rm(f"{self.base_dir}/checkpoint/invoices", True)

        dbutils.fs.mkdirs(f"{self.base_dir}/data/invoices")
        print("Done!\n")
        print("project is setup and actual process is starting now!!!!")
              
    def ingestData(self,itr):
        print(f"\tData ingestion started for itr_{itr}........",end='')
        dbutils.fs.cp(f"{self.base_dir}/datasets/invoices/invoices_{itr}.json",f"{self.base_dir}/data/invoices")
        print("Done!\n")
    
    def assertRes(self,expectedCount):
        print("\tvalidation started...",end='')
        actual_count=spark.sql("select count(*) from invoice_lineItem_table").collect()[0][0]
        assert actual_count==expectedCount , f"failed tests ..expected count is {expectedCount}"
        print("done\n")
    
    def run_tests(self):
        import time
        
        self.cleanTests()
        wc=invoiceStream()
        sQuery=wc.process()
        

        self.ingestData(1)
        time.sleep(60)
        self.assertRes(1253)

        self.ingestData(2)
        time.sleep(60)
        self.assertRes(1001)

        self.ingestData(3)
        time.sleep(60)
        self.assertRes(590)
    
        sQuery.stop()

    

    

In [0]:
obj=testInvoice()
obj.run_tests()

directory cleaning started.....Done!

project is setup and actual process is starting now!!!!
	 starting the processing of invoices .....
Done..

	Data ingestion started for itr_1........Done!

	validation started...done

	Data ingestion started for itr_2........Done!



In [0]:
def getSchema():
        return """InvoiceNumber string, CreatedTime bigint, StoreID string, PosID string, CashierID string,
                CustomerType string, CustomerCardNo string, TotalAmount double, NumberOfItems bigint, 
                PaymentMethod string, TaxableAmount double, CGST double, SGST double, CESS double, 
                DeliveryType string,
                InvoiceLineItems array<struct<ItemCode string, ItemDescription string, 
                    ItemPrice double, ItemQty bigint, TotalValue double>>
            """
df=spark.read\
            .format("json")\
            .schema(getSchema())\
            .load("dbfs:/FileStore/project1/data/invoices/invoices_1.json")
df.count()

Out[66]: 501

In [0]:
%sql
select * from invoice_lineitem_table

InvoiceNumber,CreatedTime,StoreID,PosID,CustomerType,PaymentMethod,DeliveryType,ItemCode,ItemDescription,ItemPrice,ItemQty,TotalValue
51402977,1595688900348,STR7188,POS956,PRIME,CARD,TAKEAWAY,458,Wine glass,1644.0,2,3288.0
51402977,1595688900348,STR7188,POS956,PRIME,CARD,TAKEAWAY,283,Portable Lamps,2236.0,1,2236.0
51402977,1595688900348,STR7188,POS956,PRIME,CARD,TAKEAWAY,498,Carving knifes,1424.0,2,2848.0
51402977,1595688900348,STR7188,POS956,PRIME,CARD,TAKEAWAY,523,Oil-lamp clock,1371.0,2,2742.0
91372973,1595688901219,STR8513,POS163,PRIME,CARD,HOME-DELIVERY,413,Slipcover,1896.0,1,1896.0
91372973,1595688901219,STR8513,POS163,PRIME,CARD,HOME-DELIVERY,483,Teacups and saucers,1781.0,1,1781.0
91372973,1595688901219,STR8513,POS163,PRIME,CARD,HOME-DELIVERY,583,Tab Top Curtains,1329.0,1,1329.0
91372973,1595688901219,STR8513,POS163,PRIME,CARD,HOME-DELIVERY,558,Balloon clock,1633.0,2,3266.0
93647513,1595688902254,STR8513,POS881,NONPRIME,CARD,TAKEAWAY,258,Closet,1687.0,2,3374.0
8320594,1595688902254,STR7188,POS825,PRIME,CASH,HOME-DELIVERY,288,Hutch,1812.0,2,3624.0
